# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [205]:
import pandas as pd
import numpy as np
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [97]:
stock_data = pd.read_csv('sp_500_stocks.csv')


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [98]:
from secrets import IEX_CLOUD_SANDBOX_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [99]:
payload = {"token":IEX_CLOUD_SANDBOX_API_TOKEN}
base_url = "https://sandbox.iexapis.com"
response = requests.get(f"{base_url}/stable/stock/fb/quote", payload)
data = response.json()
market_cap = data['marketCap']
price = data['latestPrice']


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [100]:
cols = ['Ticker', 'Price', 'MarketCap','SharesToBuy']
data_frame = pd.DataFrame(columns=cols)
ticker = "fb"
series = pd.Series([ticker, price, market_cap, 0], index=cols)
data_frame.append(series, ignore_index=True)

,Ticker,Price,MarketCap,SharesToBuy
0,fb,330.82,943727373223,0


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [101]:


payload = {"token":IEX_CLOUD_SANDBOX_API_TOKEN, "symbols":symbols, "types":"quote"}
response = requests.get(f"{base_url}/stable/stock/market/batch", payload)
data = response.json()


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [148]:
data_frame = pd.DataFrame(columns=cols)
def chunk_generator(big_list, chunk_size):

    for i in range(0,len(big_list), chunk_size):
        yield big_list[i:i+chunk_size]


symbol_groups = list(chunk_generator(stock_data["Ticker"], 100))

for s in symbol_groups:
    symbols = ",".join(s)
    
    payload = {"token":IEX_CLOUD_SANDBOX_API_TOKEN, "symbols":symbols, "types":"quote"}
    response = requests.get(f"{base_url}/stable/stock/market/batch", payload)
    data = response.json()
    for ticker in data:
        ticker_info = data[ticker]['quote']
#         print(ticker, ticker_info)
        market_cap = ticker_info['marketCap']
        price = ticker_info['latestPrice']

        series = pd.Series([ticker, price,market_cap, 0], index=cols)

        data_frame = data_frame.append(series, ignore_index=True)

# print(symbol_groups)



In [149]:
data_frame

,Ticker,Price,MarketCap,SharesToBuy
0,A,138.70,40932066572,0
1,AAL,23.00,14522709422,0
2,AAP,207.31,13847107927,0
3,AAPL,134.18,2237716746872,0
4,ABBV,120.12,209407029253,0
...,...,...,...,...
500,YUM,123.29,37206298888,0
501,ZBH,174.75,36437814107,0
502,ZBRA,505.46,26686658682,0
503,ZION,60.64,9827143386,0


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [150]:
#Done above

In [151]:
#Done Above

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next. 

I'll be using an equal investment startegy. i.e the position size of each stock is the same 

In [153]:
portfolio_size = input("value of portfolio")
portfolio_size = float(portfolio_size)
portfolio_size

value of portfolio1000000


1000000.0

In [191]:

n = len(data_frame.index)
position_size = portfolio_size / n

for i in range(n):
    price = data_frame.loc[i, "Price"]
    data_frame.loc[i, "SharesToBuy"] = math.floor(position_size/price)
data_frame

,Ticker,Price,MarketCap,SharesToBuy
0,A,138.70,40932066572,14
1,AAL,23.00,14522709422,86
2,AAP,207.31,13847107927,9
3,AAPL,134.18,2237716746872,14
4,ABBV,120.12,209407029253,16
...,...,...,...,...
500,YUM,123.29,37206298888,16
501,ZBH,174.75,36437814107,11
502,ZBRA,505.46,26686658682,3
503,ZION,60.64,9827143386,32


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [238]:
writer = pd.ExcelWriter("recommended_trades.xlsx", engine='xlsxwriter') 
data_frame.to_excel(writer, sheet_name="Recommended Trades", index=False) 
# index=> writing row names 


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [239]:
#Creating color scheme for formats
background_color = '#0a0a23'
font_color = "#ffffff"

#Create formats that can be assigned to columns
string_format = writer.book.add_format({
    "font_color": font_color,
    "bg_color": background_color,
    "border":1
})


dollar_format = writer.book.add_format({
    "font_color": font_color,
    "bg_color": background_color,
    "border":1,
    "num_format":'$0.00'
})


integer_format = writer.book.add_format({
    "font_color": font_color,
    "bg_color": background_color,
    "border":1,
    "num_format": '0'
})



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [240]:
# writer.sheets['Recommended Trades'].set_column('A:A', #This tells the method to apply the format to column B
#                      18, #This tells the method to apply a column width of 18 pixels
#                      string_format #This applies the format 'string_template' to the column
#                     )



This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [241]:
column_format = {
    "A":["Ticker", string_format],
    "B":["Price", dollar_format],
    "C":["Market Capitalization", dollar_format],
    "D":["Shares To Buy", integer_format]
}

for col in column_format:
    label, _format = column_format[col]
    writer.sheets["Recommended Trades"].set_column(f"{col}:{col}", 18, _format)
    writer.sheets["Recommended Trades"].write(f"{col}1", label)
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy: